In [1]:
import pandas as pd  
import numpy as np    
DATASET_FOLDER = '../dataset/'
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_test = pd.read_csv( 'sample_test.csv')
sample_test_out = pd.read_csv('sample_test_out.csv')

In [2]:
val = train[20000:30000]

In [3]:
val

,image_link,group_id,entity_name,entity_value
20000,https://m.media-amazon.com/images/I/717DLM3B5d...,529606,item_weight,7.0 ounce
20001,https://m.media-amazon.com/images/I/71by89Xk-p...,299791,item_weight,112.0 gram
20002,https://m.media-amazon.com/images/I/61ud3N26zA...,179080,voltage,48.0 volt
20003,https://m.media-amazon.com/images/I/51qmCZ9jTT...,254046,item_weight,0.128 kilogram
20004,https://m.media-amazon.com/images/I/81xKX8h5-M...,483370,item_weight,1.0 pound
...,...,...,...,...
29995,https://m.media-amazon.com/images/I/71PlsJl-mJ...,449021,item_weight,10.0 ounce
29996,https://m.media-amazon.com/images/I/71o33-+F8e...,522832,item_weight,17.99 ounce
29997,https://m.media-amazon.com/images/I/61tBLvnkHo...,267482,item_weight,3.7 pound
29998,https://m.media-amazon.com/images/I/71Qr7HYWoB...,267482,item_weight,3.7 pound


In [4]:
xval = val["image_link"].values.tolist()

In [ ]:
#from utils import download_images
#download_images(xval, "images_6"  , allow_multiprocessing=True)
#only 9858 downloaded

  0%|                                                                            | 16/10000 [01:24<12:46:01,  4.60s/it]

In [10]:
#import easyocr
# import os
# import pandas as pd
# import re
# import cv2

# # Initialize the EasyOCR Reader to use GPU (gpu=True)
# reader = easyocr.Reader(['en'], gpu=True)

# # Function to correct OCR misinterpretations based on context
# def correct_ocr_text(text):
#     # Replaces 'O' with '0' if it appears in a numerical context (surrounded by digits)
    
#     words = text.split()
#     corrected_words = []
#     for word in words:
#         # If the word contains digits, correct 'O' or 'o' to '0'
#         if re.search(r'\d', word):
#             corrected_word = re.sub(r'[Oo]', '0', word)
#         else:
#             corrected_word = word
        
#         corrected_words.append(corrected_word)

#     return ' '.join(corrected_words)

# # Function to process images and save results to CSV in batches
# def process_images_in_folder(folder_path, csv_output_file, batch_size=100):
#     results = []
    
#     # Get all image file paths in the folder
#     image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
#     total_images = len(image_files)
#     print(f"Total images to process: {total_images}")
    
#     for idx, image_file in enumerate(image_files):
#         try:
#             # Load the grayscale image from the folder
#             image_path = os.path.join(folder_path, image_file)
#             img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
#             if img is None:
#                 print(f"Error reading image {image_file}")
#                 continue

#             # Use EasyOCR to extract text
#             result = reader.readtext(img)
            
#             # Extract and concatenate text from the OCR results
#             extracted_text = ' '.join([item[1] for item in result])

#             # Correct OCR errors
#             corrected_text = correct_ocr_text(extracted_text)
            
#             results.append({'image_file': image_file, 'extracted_text': corrected_text})
            
#             # Save to CSV after every batch_size images
#             if (idx + 1) % batch_size == 0:
#                 print(f"Saving batch {idx + 1}/{total_images} to CSV...")
#                 results_df = pd.DataFrame(results, columns=['image_file', 'extracted_text'])
#                 if os.path.exists(csv_output_file):
#                     # Append without overwriting
#                     results_df.to_csv(csv_output_file, mode='a', header=False, index=False)
#                 else:
#                     results_df.to_csv(csv_output_file, index=False)
#                 results = []  # Clear the batch after saving

#         except Exception as e:
#             print(f"Error processing {image_file}: {e}")
#             results.append({'image_file': image_file, 'extracted_text': '', 'error': str(e)})
    
#     # Save any remaining results after the loop
#     if results:
#         results_df = pd.DataFrame(results, columns=['image_file', 'extracted_text'])
#         if os.path.exists(csv_output_file):
#             results_df.to_csv(csv_output_file, mode='a', header=False, index=False)
#         else:
#             results_df.to_csv(csv_output_file, index=False)
    
#     print("Processing complete!")

# # Example usage: Process images from the folder on D drive and save to CSV
# # folder_path = r'D:\your_image_folder'
# # # Replace with the actual path to your images folder on D drive
# folder_path = r'images_4'
# csv_output_file = "train_image_4.csv"

# # Process images and save results in batches of 100 images
# process_images_in_folder(folder_path, csv_output_file, batch_size=100)


In [7]:
#BEST TILL NOW
import easyocr
import os
import pandas as pd
import re
import cv2
import gc
import torch
import os

# Set PyTorch environment variable to handle memory fragmentation
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Initialize the EasyOCR Reader to use GPU (gpu=True)
reader = easyocr.Reader(['en'], gpu=True)

# Function to correct OCR misinterpretations based on context
def correct_ocr_text(text):
    words = text.split()
    corrected_words = []
    for word in words:
        if re.search(r'\d', word):
            corrected_word = re.sub(r'[Oo]', '0', word)
        else:
            corrected_word = word
        corrected_words.append(corrected_word)
    return ' '.join(corrected_words)

# Function to clear GPU memory
def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# Function to resize large images to reduce memory consumption
def resize_image(image, max_size=1024, min_size=300):
    h, w = image.shape
    if max(h, w) > max_size:
        scale = max_size / float(max(h, w))
        return cv2.resize(image, (int(w * scale), int(h * scale)))
    elif min(h, w) < min_size:
        # Skip resizing for small images to preserve detail
        return image
    return image

# Function to process images in batches with optimized GPU handling
def process_images_in_folder(folder_path, csv_output_file, batch_size=50):
    # Read the existing CSV file to determine which images have already been processed
    if os.path.exists(csv_output_file):
        processed_df = pd.read_csv(csv_output_file)
        processed_images = set(processed_df['image_file'].tolist())
    else:
        processed_images = set()

    results = []
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    total_images = len(image_files)
    print(f"Total images to process: {total_images}")

    for idx, image_file in enumerate(image_files):
        if image_file in processed_images:
            continue  # Skip already processed images

        try:
            # Load and resize the grayscale image
            image_path = os.path.join(folder_path, image_file)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
            if img is None:
                print(f"Error reading image {image_file}")
                continue

            # Resize image if too large to save memory
            img = resize_image(img, max_size=1024)

            # Use EasyOCR to extract text
            result = reader.readtext(img)
            extracted_text = ' '.join([item[1] for item in result])

            # Correct OCR errors
            corrected_text = correct_ocr_text(extracted_text)
            results.append({'image_file': image_file, 'extracted_text': corrected_text})

            # Save to CSV in batches
            if (idx + 1) % batch_size == 0:
                print(f"Saving batch {idx + 1}/{total_images} to CSV...")
                results_df = pd.DataFrame(results, columns=['image_file', 'extracted_text'])
                if os.path.exists(csv_output_file):
                    results_df.to_csv(csv_output_file, mode='a', header=False, index=False)
                else:
                    results_df.to_csv(csv_output_file, index=False)
                results = []  # Clear the batch after saving

                # Clear GPU memory after every batch
                clear_gpu_memory()

        except Exception as e:
            print(f"Error processing {image_file}: {e}")
            results.append({'image_file': image_file, 'extracted_text': '', 'error': str(e)})
            
            # Clear GPU memory in case of errors
            clear_gpu_memory()

        # Clear GPU memory more frequently after each image
        clear_gpu_memory()

    # Save remaining results
    if results:
        results_df = pd.DataFrame(results, columns=['image_file', 'extracted_text'])
        if os.path.exists(csv_output_file):
            results_df.to_csv(csv_output_file, mode='a', header=False, index=False)
        else:
            results_df.to_csv(csv_output_file, index=False)

    print("Processing complete!")

# Example usage: Process images from the folder and resume from where it stopped
folder_path = r'images_7'
csv_output_file = "train_image_7.csv"

# Resume processing images and save results in batches of 50 images
process_images_in_folder(folder_path, csv_output_file, batch_size=50)


C:\Users\ashil\AppData\Roaming\Python\Python312\site-packages\easyocr\detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.

Total images to process: 9870
Saving batch 50/9870 to CSV...
Saving batch 100/9870 to CSV...
Saving batch 150/9870 to CSV...
Saving batch 200/9870 to CSV...
Saving batch 250/9870 to CSV...
Saving batch 300/9870 to CSV...
Saving batch 350/9870 to CSV...
Saving batch 400/9870 to CSV...
Saving batch 450/9870 to CSV...
Saving batch 500/9870 to CSV...
Saving batch 550/9870 to CSV...
Saving batch 600/9870 to CSV...
Saving batch 650/9870 to CSV...
Saving batch 700/9870 to CSV...
Saving batch 750/9870 to CSV...
Saving batch 800/9870 to CSV...
Saving batch 850/9870 to CSV...
Saving batch 900/9870 to CSV...
Saving batch 950/9870 to CSV...
Saving batch 1000/9870 to CSV...
Saving batch 1050/9870 to CSV...
Saving batch 1100/9870 to CSV...
Saving batch 1150/9870 to CSV...
Saving batch 1200/9870 to CSV...
Saving batch 1250/9870 to CSV...
Saving batch 1300/9870 to CSV...
Saving batch 1350/9870 to CSV...
Saving batch 1400/9870 to CSV...
Saving batch 1450/9870 to CSV...
Saving batch 1500/9870 to CSV...


In [ ]:
#BEST TILL NOW
import easyocr
import os
import pandas as pd
import re
import cv2
import gc
import torch
import os

# Set PyTorch environment variable to handle memory fragmentation
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Initialize the EasyOCR Reader to use GPU (gpu=True)
reader = easyocr.Reader(['en'], gpu=True)

# Function to correct OCR misinterpretations based on context
def correct_ocr_text(text):
    words = text.split()
    corrected_words = []
    for word in words:
        if re.search(r'\d', word):
            corrected_word = re.sub(r'[Oo]', '0', word)
        else:
            corrected_word = word
        corrected_words.append(corrected_word)
    return ' '.join(corrected_words)

# Function to clear GPU memory
def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# Function to resize large images to reduce memory consumption
def resize_image(image, max_size=1024, min_size=300):
    h, w = image.shape
    if max(h, w) > max_size:
        scale = max_size / float(max(h, w))
        return cv2.resize(image, (int(w * scale), int(h * scale)))
    elif min(h, w) < min_size:
        # Skip resizing for small images to preserve detail
        return image
    return image

# Function to process images in batches with optimized GPU handling
def process_images_in_folder(folder_path, csv_output_file, batch_size=50):
    # Read the existing CSV file to determine which images have already been processed
    if os.path.exists(csv_output_file):
        processed_df = pd.read_csv(csv_output_file)
        processed_images = set(processed_df['image_file'].tolist())
    else:
        processed_images = set()

    results = []
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    total_images = len(image_files)
    print(f"Total images to process: {total_images}")

    for idx, image_file in enumerate(image_files):
        if image_file in processed_images:
            continue  # Skip already processed images

        try:
            # Load and resize the grayscale image
            image_path = os.path.join(folder_path, image_file)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
            if img is None:
                print(f"Error reading image {image_file}")
                continue

            # Resize image if too large to save memory
            img = resize_image(img, max_size=1024)

            # Use EasyOCR to extract text
            result = reader.readtext(img)
            extracted_text = ' '.join([item[1] for item in result])

            # Correct OCR errors
            corrected_text = correct_ocr_text(extracted_text)
            results.append({'image_file': image_file, 'extracted_text': corrected_text})

            # Save to CSV in batches
            if (idx + 1) % batch_size == 0:
                print(f"Saving batch {idx + 1}/{total_images} to CSV...")
                results_df = pd.DataFrame(results, columns=['image_file', 'extracted_text'])
                if os.path.exists(csv_output_file):
                    results_df.to_csv(csv_output_file, mode='a', header=False, index=False)
                else:
                    results_df.to_csv(csv_output_file, index=False)
                results = []  # Clear the batch after saving

                # Clear GPU memory after every batch
                clear_gpu_memory()

        except Exception as e:
            print(f"Error processing {image_file}: {e}")
            results.append({'image_file': image_file, 'extracted_text': '', 'error': str(e)})
            
            # Clear GPU memory in case of errors
            clear_gpu_memory()

        # Clear GPU memory more frequently after each image
        clear_gpu_memory()

    # Save remaining results
    if results:
        results_df = pd.DataFrame(results, columns=['image_file', 'extracted_text'])
        if os.path.exists(csv_output_file):
            results_df.to_csv(csv_output_file, mode='a', header=False, index=False)
        else:
            results_df.to_csv(csv_output_file, index=False)

    print("Processing complete!")

# Example usage: Process images from the folder and resume from where it stopped
folder_path = r'images_10'
csv_output_file = "train_image_10.csv"

# Resume processing images and save results in batches of 50 images
process_images_in_folder(folder_path, csv_output_file, batch_size=50)


Total images to process: 9895
Saving batch 50/9895 to CSV...
Saving batch 100/9895 to CSV...
Saving batch 150/9895 to CSV...
Saving batch 200/9895 to CSV...
Saving batch 250/9895 to CSV...
Saving batch 300/9895 to CSV...
Saving batch 350/9895 to CSV...
Saving batch 400/9895 to CSV...
Saving batch 450/9895 to CSV...
Saving batch 500/9895 to CSV...
Saving batch 550/9895 to CSV...
Saving batch 600/9895 to CSV...
Saving batch 650/9895 to CSV...
Saving batch 700/9895 to CSV...
Saving batch 750/9895 to CSV...
Saving batch 800/9895 to CSV...
Saving batch 850/9895 to CSV...
Saving batch 900/9895 to CSV...
Saving batch 950/9895 to CSV...
Saving batch 1000/9895 to CSV...
Saving batch 1050/9895 to CSV...
Saving batch 1100/9895 to CSV...
Saving batch 1150/9895 to CSV...
Saving batch 1200/9895 to CSV...
Saving batch 1250/9895 to CSV...
Saving batch 1300/9895 to CSV...
Saving batch 1350/9895 to CSV...
Saving batch 1400/9895 to CSV...
Saving batch 1450/9895 to CSV...
Saving batch 1500/9895 to CSV...


In [ ]:
#BEST TILL NOW
import easyocr
import os
import pandas as pd
import re
import cv2
import gc
import torch
import os

# Set PyTorch environment variable to handle memory fragmentation
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Initialize the EasyOCR Reader to use GPU (gpu=True)
reader = easyocr.Reader(['en'], gpu=True)

# Function to correct OCR misinterpretations based on context
def correct_ocr_text(text):
    words = text.split()
    corrected_words = []
    for word in words:
        if re.search(r'\d', word):
            corrected_word = re.sub(r'[Oo]', '0', word)
        else:
            corrected_word = word
        corrected_words.append(corrected_word)
    return ' '.join(corrected_words)

# Function to clear GPU memory
def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# Function to resize large images to reduce memory consumption
def resize_image(image, max_size=1024, min_size=300):
    h, w = image.shape
    if max(h, w) > max_size:
        scale = max_size / float(max(h, w))
        return cv2.resize(image, (int(w * scale), int(h * scale)))
    elif min(h, w) < min_size:
        # Skip resizing for small images to preserve detail
        return image
    return image

# Function to process images in batches with optimized GPU handling
def process_images_in_folder(folder_path, csv_output_file, batch_size=50):
    # Read the existing CSV file to determine which images have already been processed
    if os.path.exists(csv_output_file):
        processed_df = pd.read_csv(csv_output_file)
        processed_images = set(processed_df['image_file'].tolist())
    else:
        processed_images = set()

    results = []
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    total_images = len(image_files)
    print(f"Total images to process: {total_images}")

    for idx, image_file in enumerate(image_files):
        if image_file in processed_images:
            continue  # Skip already processed images

        try:
            # Load and resize the grayscale image
            image_path = os.path.join(folder_path, image_file)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
            if img is None:
                print(f"Error reading image {image_file}")
                continue

            # Resize image if too large to save memory
            img = resize_image(img, max_size=1024)

            # Use EasyOCR to extract text
            result = reader.readtext(img)
            extracted_text = ' '.join([item[1] for item in result])

            # Correct OCR errors
            corrected_text = correct_ocr_text(extracted_text)
            results.append({'image_file': image_file, 'extracted_text': corrected_text})

            # Save to CSV in batches
            if (idx + 1) % batch_size == 0:
                print(f"Saving batch {idx + 1}/{total_images} to CSV...")
                results_df = pd.DataFrame(results, columns=['image_file', 'extracted_text'])
                if os.path.exists(csv_output_file):
                    results_df.to_csv(csv_output_file, mode='a', header=False, index=False)
                else:
                    results_df.to_csv(csv_output_file, index=False)
                results = []  # Clear the batch after saving

                # Clear GPU memory after every batch
                clear_gpu_memory()

        except Exception as e:
            print(f"Error processing {image_file}: {e}")
            results.append({'image_file': image_file, 'extracted_text': '', 'error': str(e)})
            
            # Clear GPU memory in case of errors
            clear_gpu_memory()

        # Clear GPU memory more frequently after each image
        clear_gpu_memory()

    # Save remaining results
    if results:
        results_df = pd.DataFrame(results, columns=['image_file', 'extracted_text'])
        if os.path.exists(csv_output_file):
            results_df.to_csv(csv_output_file, mode='a', header=False, index=False)
        else:
            results_df.to_csv(csv_output_file, index=False)

    print("Processing complete!")

# Example usage: Process images from the folder and resume from where it stopped
folder_path = r'images_11'
csv_output_file = "train_image_11.csv"

# Resume processing images and save results in batches of 50 images
process_images_in_folder(folder_path, csv_output_file, batch_size=50)


In [13]:
val = train[70000:80000]

In [14]:
xval = val["image_link"].values.tolist()

In [15]:
from utils import download_images
download_images(xval, "images_11"  , allow_multiprocessing=True)


100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [01:29<00:00, 112.18it/s]


In [4]:
import pandas as pd
import re

# Load the CSV file
file_path = "train_image_4.csv"  # Replace with the actual path to your CSV file
df = pd.read_csv(file_path)

# Define a mapping for unit abbreviations
unit_variants = {
    "centimetre": ["cm", "centimetre"],
    "foot": ["ft", "foot"],
    "millimetre": ["mm", "millimetre"],
    "metre": ["m", "metre"],
    "inch": ["in", "inch"],
    "yard": ["yd", "yard"],
    "milligram": ["mg", "milligram"],
    "kilogram": ["kg", "kilogram"],
    "microgram": ["µg", "microgram"],
    "gram": ["g", "gram"],
    "ounce": ["oz", "ounce"],
    "ton": ["ton"],
    "pound": ["lb", "pound"],
    "millivolt": ["mv", "millivolt"],
    "kilovolt": ["kv", "kilovolt"],
    "volt": ["v", "volt"],
    "kilowatt": ["kw", "kilowatt"],
    "watt": ["w", "watt"],
    "cubic foot": ["cubic foot"],
    "microlitre": ["µl", "microlitre"],
    "cup": ["cup"],
    "fluid ounce": ["fl oz", "fluid ounce"],
    "centilitre": ["cl", "centilitre"],
    "imperial gallon": ["imperial gallon"],
    "pint": ["pt", "pint"],
    "decilitre": ["dl", "decilitre"],
    "litre": ["l", "litre"],
    "millilitre": ["ml", "millilitre"],
    "quart": ["qt", "quart"],
    "cubic inch": ["cubic inch"],
    "gallon": ["gal", "gallon"]
}

# Create a regex pattern for units with case insensitivity and variations
unit_pattern = "|".join([f"{abbrev}" for unit, abbrevs in unit_variants.items() for abbrev in abbrevs])

# Define regex to handle commas in numbers and case insensitivity for units
def extract_values(text):
    # Match numbers with optional commas and decimals, followed by a unit
    matches = re.findall(rf'(\d{{1,3}}(?:,\d{{3}})*(?:\.\d+)?)\s*({unit_pattern})', text, re.IGNORECASE)
    # Remove commas from the numeric part
    matches = [(re.sub(r',', '', num), unit) for num, unit in matches]
    return matches

# Apply the extraction function to the 'extracted_text' column
df['extracted_values'] = df['extracted_text'].apply(extract_values)

# Display the DataFrame with the extracted values#


TypeError: expected string or bytes-like object, got 'float'

In [ ]:
df

In [ ]:
xvp = val["image_link"].str.replace("https://m.media-amazon.com/images/I/" , "")
val["image_link"] = xvp

In [ ]:
target_list = []
image_path_list = df["image_file"].values.tolist()
update_images = []
for v in image_path_list:
    update_images.append(v)
    #x = val[val["image_link"] == v]
    #if x.shape[0] == 1:
        #update_images.append(val[val["image_link"] == v]["group_id"].values.tolist()[0])
print(update_images)

In [ ]:
filtered_df = val[val['image_link'].isin(update_images)]

In [ ]:
text_corpus = df["extracted_text"].values.tolist()

In [ ]:
def extract_numbers(input_string):
    numbers = ''.join([char for char in input_string if char.isdigit()])
    return numbers

In [ ]:
filtered_df

In [ ]:
df

In [ ]:
alias_map = {
    'centimetre': ['cm', 'centimetre', 'centimeter', 'centimetres', 'centimeters'],
    'foot': ['ft', 'foot', 'feet'],
    'inch': ['in', 'inch', 'inches'],
    'metre': ['m', 'metre', 'meter', 'metres', 'meters'],
    'millimetre': ['mm', 'millimetre', 'millimeter', 'millimetres', 'millimeters'],
    'yard': ['yd', 'yard', 'yards'],

    'gram': ['g', 'gm', 'gram', 'grams'],
    'kilogram': ['kg', 'kilogram', 'kilograms'],
    'microgram': ['mcg', 'µg', 'microgram', 'micrograms'],
    'milligram': ['mg', 'milligram', 'milligrams'],
    'ounce': ['oz', 'ounce', 'ounces'],
    'pound': ['lb', 'lbs', 'pound', 'pounds'],
    'ton': ['ton', 'tons', 'tonne', 'tonnes'],

    'kilovolt': ['kV', 'kilovolt', 'kilovolts' , "kv"],
    'millivolt': ['mV', 'millivolt', 'millivolts' , "mv"],
    'volt': ['V', 'volt', 'volts'],

    'kilowatt': ['kW', 'kilowatt', 'kilowatts'],
    'watt': ['W', 'watt', 'watts'],

    'centilitre': ['cl', 'centilitre', 'centilitres'],
    'cubic foot': ['cu ft', 'cubic foot', 'cubic feet'],
    'cubic inch': ['cu in', 'cubic inch', 'cubic inches'],
    'cup': ['cup', 'cups'],
    'decilitre': ['dl', 'decilitre', 'decilitres'],
    'fluid ounce': ['fl oz', 'fluid ounce', 'fluid ounces'],
    'gallon': ['gal', 'gallon', 'gallons'],
    'imperial gallon': ['imp gal', 'imperial gallon', 'imperial gallons'],
    'litre': ['l', 'litre', 'liter', 'litres', 'liters'],
    'microlitre': ['µl', 'microlitre', 'microliter', 'microlitres', 'microliters'],
    'millilitre': ['ml', 'millilitre', 'milliliter', 'millilitres', 'milliliters'],
    'pint': ['pt', 'pint', 'pints'],
    'quart': ['qt', 'quart', 'quarts']
}

lsit = []
for key in alias_map:
    lsit.extend(alias_map[key])

In [ ]:
def check_unit(strg):
    if strg in lsit:
        return True
    else:
        return False

In [ ]:
true_counter = 0
false_counter = 0
flg = 1
for idx , img in enumerate(filtered_df["image_link"].values.tolist()):
    text  = df[df["image_file"] == img]["extracted_values"].values.tolist()[0]
    acc_value = []
    acc_unit = []
    for t in text:
        acc_unit.append(t[1])
        acc_value.append(t[0])
    value_x , unit_x = filtered_df[filtered_df["image_link"] == img]["entity_value"].values[0].split(" ")
    value_x = float(value_x)
    for idx , a_v in enumerate(acc_value):
        x = float(a_v)
        y = acc_unit[idx]
        if x == value_x and check_unit(y):
            true_counter += 1
            flg = 0
            break
        else:
            flg = 1
    false_counter += flg
            
        

In [ ]:
print(true_counter)
print(false_counter)